# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

GOOGLE_API_KEY=****re4I
LANGSMITH_TRACING=true
LANGSMITH_API_KEY=****ad5c
LANGSMITH_PROJECT=****ials


## Human👨‍💻 and AI 🤖 Messages

In [7]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
model = "google_genai:gemini-2.5-flash"
agent = create_agent(
    model=model, 
    system_prompt="You are a full-stack comedian"
)

In [8]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [9]:
print(result["messages"][-1].content)

Hello there!

My front-end is rendering perfectly, my back-end is responding with a solid 200 OK, and my database is querying for more coffee. So, I'd say I'm in a pretty stable production environment!

How about you? Any unexpected `null` values in your day?


In [10]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [11]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hello there!

My front-end is rendering perfectly, my back-end is responding with a solid 200 OK, and my database is querying for more coffee. So, I'd say I'm in a pretty stable production environment!

How about you? Any unexpected `null` values in your day?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [12]:
agent = create_agent(
    model=model,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [13]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Green diamond, sun's slow arc.
A thrown sphere, then a crack.
Dust rises, bases run.
Fielders wait, day is spun.


#### Dictionaries

In [14]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Gunshot explodes,
Muscles surge, a blur of speed,
Finish tape breaks.


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [15]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [17]:
agent = create_agent(
    model=model,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [18]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Green fields, open skies,
Sweat and cheers, a thrilling game,
Victory's sweet taste.


In [19]:
result["messages"][-1].content

"Green fields, open skies,\nSweat and cheers, a thrilling game,\nVictory's sweet taste."

In [20]:
print(len(result["messages"]))

4


In [21]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================

[{'type': 'text', 'text': "Green fields, open skies,\nSweat and cheers, a thrilling game,\nVictory's sweet taste.", 'extras': {'signature': 'CsYBAXLI2nyOjFPNB3igvQ+lLfBorPnMFKM9ieC6JWLjLzl41AVvu6xqzQ8KdyyfjJI0EuLiMN5QNwYMOvYSLtEY4ZCUje8GlW/INNkIpTIViHO46RQKympPe2iZ4yixHpVdQXL/NXWOIuip5sus8D6B4LiuBFpMbysid3aRmgFBez/hnBA+Pe9U8BMSnXN2+t6Np4YnxtHSoS7amTFpOTal2bC85k1EhWZGyyL0t5H6moB/8yb51CKbwY7Xlp1O3w5wq6A3VEdQ'}}]
Tool Calls:
  check_haiku_lines (b00dfe87-2cad-415c-a04a-bc7d7712c40c)
 Call ID: b00dfe87-2cad-415c-a04a-bc7d7712c40c
  Args:
    text: Green fields, open skies,
Sweat and cheers, a thrilling game,
Victory's sweet taste.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai M

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [28]:
result['messages'][0].content

'Please write me a poem'

You can select just the last message, and you can see where the final message is coming from.

In [29]:
result["messages"][-1]

AIMessage(content="Green fields, open skies,\nSweat and cheers, a thrilling game,\nVictory's sweet taste.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--d93ee19f-f5ac-4b65-91fe-9a682f1f89f2-0', usage_metadata={'input_tokens': 152, 'output_tokens': 23, 'total_tokens': 175, 'input_token_details': {'cache_read': 0}})

In [30]:
result["messages"][-1].usage_metadata

{'input_tokens': 152,
 'output_tokens': 23,
 'total_tokens': 175,
 'input_token_details': {'cache_read': 0}}

In [31]:
result["messages"][-1].response_metadata

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.5-flash',
 'safety_ratings': [],
 'model_provider': 'google_genai'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [ ]:
agent = create_agent(
    model=model,
    tools=[check_haiku_lines],
    system_prompt="You are a poetic programmer in python. You always describe a problem of a program in poetic ways.",
)

In [32]:
result = agent.invoke({"messages": "Describe a calculators logic."})

checking haiku, it has 3 lines:
 Numbers in, then out,
Logic gates will hum and whir,
Answer is revealed.


In [33]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Describe a calculators logic.
================================== Ai Message ==================================

[{'type': 'text', 'text': 'Numbers in, then out,\nLogic gates will hum and whir,\nAnswer is revealed.', 'extras': {'signature': 'CrwEAXLI2nxHjdnrqOH1CilP3MRPTvKj0G2WI1sMcc84+vOZwPJv785EgkMWenYb82uWR5ZNBTkPHCcDxLizpNf09t3OcjyWSmMptxvk9k5Y/I78u5h28iiBoMM4tDv85+K67ZXpBhOj6N8+2qL7WbaQ6bvSSq29mUhhSejOT7f29jbVj6yYc7GZIWmwHwwzTDZvLrOQVAQ8VaufxXb0Vjm3LisSQej3db+5ypGp6F1RaEXr0gXG/SR0X/Z9bE81Z2N3FAK7L91hU4RR7BcdFRd4rxx+FgR3FfuCFUKwIDow5wteHaKPhUs+DRlKJcVLZbu6tGWaPX9kU3+kvWQYaV1wR5BcSkBqixcu7DeHGVdb6/v9/wWjdFd+J5tfMm/3G+2q6T6bfRwkVsE0TeX4OBRlYzidCdE4KCNG1nLQwzAMnxnZBN2MdamTbjtVGhFh4/FeZyetznaFsuSUY+NRe6hZkcjMPLYAhsE6MdyWINQUwvCAiSapbr3KfN8bcl5hR1lp9P5KN0fiMK9dqu0UVThkUCdmxZtTN9JlZCJhAbBFbDXYaiCLRNI7bBMeaQBaT2l+iM4smeitPF3S4q1q9DBGzSL9WuZa7HbYsw9JDpEfyawc+KePDEEOYiraNsvFto/BEhiMILn8FqJcZFZ2mIfcB+nZUu8mtSFlA